# Initial AI Agent PPO w/o CNN (working) MR2 HS Disc

In [1]:
import torch

if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available")

GPU is available


In [2]:
from myRTClass import MyGranTurismoRTGYM, DEFAULT_CONFIG_DICT
import numpy as np
import gymnasium
from time import sleep

my_config = DEFAULT_CONFIG_DICT
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.1
my_config["start_obs_capture"] = 0.1
my_config["time_step_timeout_factor"] = 4.0
my_config["ep_max_length"] = 2000
my_config["act_buf_len"] = 4
my_config["reset_act_buf"] = True
my_config["benchmark"] = False
my_config["benchmark_polyak"] = 0.2

my_config["interface_kwargs"] = {
  'debugFlag': False, # do not use render() while True
  'discreteAccel' : True,
  'accelAndBrake' : False,
  'discSteer' : True,
  'contAccelOnly' : False,
  'discAccelOnly' : False,
  'modelMode': 13,
  #  [42, 42, K], [84, 84, K], [10, 10, K], [240, 320, K] and  [480, 640, K]
  'imageWidth' : 42,
  'imageHeight' : 42,
  'trackChoice' : 2, # 1 is High Speed Ring, 2 is 0-400m, 
}

In [3]:
def env_creator(env_config):
    env = gymnasium.make("real-time-gym-v1", config=env_config)
    return env  # return an env instance

from ray.tune.registry import register_env
register_env("gt-rtgym-env-v1", lambda config: env_creator(my_config)) # better way

In [4]:
import ray
ray.shutdown()
ray.init()

2023-10-09 07:28:02,654	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.13
Ray version:,2.6.3
Dashboard:,http://127.0.0.1:8265


In [5]:
from ray.rllib.algorithms.impala.impala import ImpalaConfig
config = (
    ImpalaConfig()
    .resources(
        num_gpus=1
        )
    .rollouts(
        num_rollout_workers=1,
        enable_connectors=True,
        batch_mode="truncate_episodes",
        )
    .framework(
        framework="torch",
        )
    .environment(
        env="gt-rtgym-env-v1",
        disable_env_checking=True,
        render_env=False,
        )
    .training(
        #train_batch_size=128,
        )
)

#APPOConfig.framework("")

2023-10-09 07:28:05,863	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


In [6]:
import json

class CustomEncoder(json.JSONEncoder):
    def default(self, obj):
        try:
            json.dumps(obj)
        except TypeError:
            return "Not serializable object!"

        return obj

config_dict = config.to_dict()

print(json.dumps(config_dict, sort_keys=True, indent=4, cls=CustomEncoder))

{
    "_AlgorithmConfig__prior_exploration_config": null,
    "_disable_action_flattening": false,
    "_disable_execution_plan_api": true,
    "_disable_initialize_loss_from_dummy_batch": false,
    "_disable_preprocessor_api": false,
    "_enable_learner_api": false,
    "_enable_rl_module_api": false,
    "_fake_gpus": false,
    "_is_atari": null,
    "_learner_class": null,
    "_lr_vf": 0.0005,
    "_minibatch_size": "auto",
    "_separate_vf_optimizer": false,
    "_tf_policy_handles_more_than_one_loss": true,
    "action_mask_key": "action_mask",
    "action_space": null,
    "actions_in_input_normalized": false,
    "after_train_step": null,
    "algorithm_config_overrides_per_module": {},
    "always_attach_evaluation_results": false,
    "auto_wrap_old_gym_envs": true,
    "batch_mode": "truncate_episodes",
    "broadcast_interval": 1,
    "callbacks": "Not serializable object!",
    "checkpoint_trainable_policies_only": false,
    "clip_actions": false,
    "clip_rewards": 

In [7]:
algo = config.build()

c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch_tb\lib\site-packages\ray\rllib\algorithms\algorithm.py:484: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch_tb\lib\site-packages\ray\tune\logger\unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch_tb\lib\site-packages\ray\tune\logger\unified.py:53: RayDeprecationWarning

(RolloutWorker pid=19440) GT Real Time instantiated
(RolloutWorker pid=19440) GT AI Server instantiated for rtgym
(RolloutWorker pid=19440) starting up on localhost port 9999
(RolloutWorker pid=19440) Waiting for a connection


(RolloutWorker pid=19440) 2023-10-09 07:28:17,343	WARNING deprecation.py:50 -- DeprecationWarning: `LearningRateSchedule` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=19440) 2023-10-09 07:28:17,343	WARNING deprecation.py:50 -- DeprecationWarning: `EntropyCoeffSchedule` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=19440) 2023-10-09 07:28:17,346	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.fcnet.FullyConnectedNetwork` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=19440) 2023-10-09 07:28:17,346	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.torch_modelv2.TorchModelV2` has been deprecated. Use `ray.rllib.core.rl_module.rl_module.RLModule` instead. This will raise an error in the future!
(RolloutWorker pid=19440) 2023-10-09 07:28:17,376	WARNING deprecation.py:50 -- DeprecationWarning: `StochasticSampling` has been deprecated. This will

(RolloutWorker pid=19440) Connection from ('127.0.0.1', 61709)
(RolloutWorker pid=19440) Discrete Accel or Brake (cannot left foot) and Discrete Steering 2


2023-10-09 07:28:17,628	WARNING deprecation.py:50 -- DeprecationWarning: `StochasticSampling` has been deprecated. This will raise an error in the future!
2023-10-09 07:28:17,629	WARNING deprecation.py:50 -- DeprecationWarning: `Exploration` has been deprecated. This will raise an error in the future!
2023-10-09 07:28:17,630	WARNING deprecation.py:50 -- DeprecationWarning: `Random` has been deprecated. This will raise an error in the future!
2023-10-09 07:28:17,630	WARNING deprecation.py:50 -- DeprecationWarning: `ValueNetworkMixin` has been deprecated. This will raise an error in the future!


In [ ]:
algo.get_policy().model

In [ ]:
N = 50000

for n in range(N):
    result = algo.train()
    print("Loop: ", n)
    if n % 10 == 0:
        print("Saved", n)
        algo.save()
        
algo.save()
